In [1]:
%load_ext autoreload
%autoreload 2
   
from clfm import clfm
import matplotlib.pyplot as plt

In [2]:
path_database = '/media/elea/Elements/output_test'
type_alignment = 'water'
comparer = clfm.CLFMProject(path_database, type_alignment)
comparer.prepare_imgs()

# Run the different methods and save the coordinates

### 1. SIFT + MNN

In [30]:
sift_mnn_matcher = clfm.Matcher('sift', 'mnn', repetitions=comparer.repetitions)
sift_mnn_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.33s/it]

Processing time: 1.11 ± 0.00 s


### 2. SiLK + MNN

In [31]:
# silk_mnn_matcher = clfm.Matcher('silk', 'mnn', repetitions=comparer.repetitions)
# silk_mnn_matcher.run(comparer.parameters_dict, comparer.images)

### 3. Superpoint + MNN

In [32]:
sp_mnn_matcher = clfm.Matcher('superpoint', 'mnn', repetitions=comparer.repetitions)
sp_mnn_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]

Processing time: 1.82 ± 0.00 s


### 4. SIFT + OANet

In [33]:
sift_oanet_matcher = clfm.Matcher('sift', 'oanet', repetitions=comparer.repetitions)
sift_oanet_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.74s/it]

Processing time: 2.52 ± 0.00 s


### 5. SiLK + OANet

In [34]:
# silk_oanet_matcher = clfm.Matcher('silk', 'oanet', repetitions=comparer.repetitions)
# silk_oanet_matcher.run(comparer.parameters_dict, comparer.images)

### 6. Superpoint + OANet

In [35]:
sp_oanet_matcher = clfm.Matcher('superpoint', 'oanet', repetitions=comparer.repetitions)
sp_oanet_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.57s/it]

Processing time: 3.34 ± 0.00 s


### 7. SP + SG

In [36]:
sp_sg_matcher = clfm.Matcher('superpoint', 'superglue', repetitions=comparer.repetitions)
sp_sg_matcher.run(comparer.parameters_dict, comparer.images)

Local feature matching:   0%|                                                                                                                                           | 0/1 [00:00<?, ?it/s]

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
==> Processing image directory input: /home/elea/Documents/HORAO/repositories/CLFM/tmp


Local feature matching: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.10s/it]

Processing time: 3.87 ± 0.00 s


### 8. Omniglue

In [37]:
"""omniglue_matcher = matcher.Matcher('omniglue', 'omniglue', repetitions=comparer.repetitions)
omniglue_matcher.run(comparer.parameters_dict, comparer.images)"""

"omniglue_matcher = matcher.Matcher('omniglue', 'omniglue', repetitions=comparer.repetitions)\nomniglue_matcher.run(comparer.parameters_dict, comparer.images)"

# Align the images using bUnwarpJ and the previously obtained coordinates

In [26]:
combinations = (
    ('sift', 'mnn'),
    # ('silk', 'mnn'),
    ('superpoint', 'mnn'),
    ('sift', 'oanet'),
    # ('silk', 'oanet'),
    ('superpoint', 'oanet'),
    ('superpoint', 'superglue'),
    # ('omniglue', 'omniglue')
)

In [49]:
from clfm import register, metrics
if type_alignment != 'histology':
    register.register_images(
        comparer.parameters_dict,
        ['gt']
    )
    pass
else:
    combinations = (*combinations, ('manual'))

Aligning images:   0%|                                                                                                                                                  | 0/1 [00:00<?, ?it/s][ERROR] Cannot create plugin: org.scijava.plugins.scripting.javascript.JavaScriptScriptLanguage


Aligning images in /home/elea/Documents/HORAO/repositories/CLFM/tmp


Aligning images: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:37<00:00, 37.07s/it]

Stacking images: ./tmp/registered_img_x.tif and ./tmp/registered_img_y.tif
Image shapes: img_x: (3, 388, 516), img_y: (3, 388, 516)


In [56]:
rmses, precisions = metrics.find_rmse(
    comparer.parameters_dict,
    comparer.images,
    comparer.type_alignment,
    combinations
)

Calculating RMSE: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.24s/it]


In [57]:
rmses_agg, precisions_agg = metrics.aggregate_and_bootstrap_metrics(
    comparer.parameters_dict,
    combinations,
    type_alignment,
    num_samples=1000
)

metrics.print_bootstrap_results(
    rmses_agg,
    precisions_agg
)

Bootstrapping metrics: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 15.17it/s]

Combination               |   RMSE(px) ± error      |   RMSE(mm) ± error      |  Prec(%) ± error   
-----------------------------------------------------------------------------------------------
sift + mnn                |     228.58 ± 11.67      |       7.77 ± 0.40       |     5.78 ± 2.36    
superpoint + mnn          |     116.35 ± 16.81      |       3.96 ± 0.57       |    39.65 ± 6.28    
sift + oanet              |       9.37 ± 1.64       |       0.32 ± 0.06       |    44.77 ± 10.11   
superpoint + oanet        |       6.83 ± 1.02       |       0.23 ± 0.03       |    60.59 ± 7.79    
superpoint + superglue    |       8.90 ± 0.67       |       0.30 ± 0.02       |    36.94 ± 5.31    
